# Multicurve Bootstrapping

This notebook replicates the QuantLib `MulticurveBootstrapping` example: build an OIS
discounting curve and a Euribor 6M forecasting curve, then price spot and forward
swaps under the dual-curve framework.

In [ ]:
import pyquantlib as ql
from datetime import date

print(f"PyQuantLib {ql.__version__} (QuantLib {ql.QL_VERSION})")

## Setup

In [ ]:
calendar = ql.TARGET()
todays_date = date(2012, 12, 11)
ql.Settings.evaluationDate = todays_date

fixing_days = 2
settlement_date = calendar.advance(todays_date, fixing_days, ql.Days)
settlement_date = calendar.adjust(settlement_date)

print(f"Today:      {todays_date}")
print(f"Settlement: {settlement_date}")

## OIS Discounting Curve

Bootstrap from overnight deposit rates, dated OIS, and long-term OIS swap rates
using `Eonia` as the overnight index and cubic discount factor interpolation.

In [ ]:
ts_day_counter = ql.Actual365Fixed()
deposit_dc = ql.Actual360()
eonia = ql.Eonia()

ois_helpers = []

# Overnight deposits
for sd, rate in [(0, 0.0004), (1, 0.0004), (2, 0.0004)]:
    h = ql.DepositRateHelper(
        rate, ql.Period(1, ql.Days), sd,
        calendar, ql.Following, False, deposit_dc,
    )
    ois_helpers.append(h)

# Short-term OIS
for tenor, rate in [("1W", 0.00070), ("2W", 0.00069), ("3W", 0.00078), ("1M", 0.00074)]:
    h = ql.OISRateHelper(2, ql.Period(tenor), rate, eonia)
    ois_helpers.append(h)

# Dated OIS
dated_ois_data = [
    (date(2013, 1, 16), date(2013, 2, 13),  0.000460),
    (date(2013, 2, 13), date(2013, 3, 13),  0.000160),
    (date(2013, 3, 13), date(2013, 4, 10), -0.000070),
    (date(2013, 4, 10), date(2013, 5, 8),  -0.000130),
    (date(2013, 5, 8),  date(2013, 6, 12), -0.000140),
]
for start, end, rate in dated_ois_data:
    h = ql.OISRateHelper(start, end, rate, eonia)
    ois_helpers.append(h)

# Long-term OIS
long_ois_data = [
    ("15M", 0.00002), ("18M", 0.00008), ("21M", 0.00021),
    ("2Y",  0.00036), ("3Y",  0.00127), ("4Y",  0.00274),
    ("5Y",  0.00456), ("6Y",  0.00647), ("7Y",  0.00827),
    ("8Y",  0.00996), ("9Y",  0.01147), ("10Y", 0.01280),
    ("11Y", 0.01404), ("12Y", 0.01516), ("15Y", 0.01764),
    ("20Y", 0.01939), ("25Y", 0.02003), ("30Y", 0.02038),
]
for tenor, rate in long_ois_data:
    h = ql.OISRateHelper(2, ql.Period(tenor), rate, eonia)
    ois_helpers.append(h)

ois_curve = ql.PiecewiseCubicDiscount(
    todays_date, ois_helpers, ts_day_counter)
ois_curve.enableExtrapolation()

discounting_handle = ql.RelinkableYieldTermStructureHandle(ois_curve)

print(f"OIS discounting curve built with {len(ois_helpers)} helpers")
print(f"  (3 deposits + 4 short OIS + {len(dated_ois_data)} dated OIS + {len(long_ois_data)} long OIS)")

## Euribor 6M Forecasting Curve

Bootstrap from a 6M deposit, FRA rates, and swap rates. The swap helpers use
the OIS curve for discounting (dual-curve bootstrapping).

In [ ]:
euribor6m = ql.Euribor6M()
euribor6m_helpers = []

# 6M deposit
d6m = ql.DepositRateHelper(
    0.00312, ql.Period("6M"), 3,
    calendar, ql.Following, False, deposit_dc,
)
euribor6m_helpers.append(d6m)

# FRAs (months to start, rate)
fra_data = [
    ( 1, 0.002930), ( 2, 0.002720), ( 3, 0.002600), ( 4, 0.002560),
    ( 5, 0.002520), ( 6, 0.002480), ( 7, 0.002540), ( 8, 0.002610),
    ( 9, 0.002670), (10, 0.002790), (11, 0.002910), (12, 0.003030),
    (13, 0.003180), (14, 0.003350), (15, 0.003520), (16, 0.003710),
    (17, 0.003890), (18, 0.004090),
]
for months, rate in fra_data:
    h = ql.FraRateHelper(rate, months, euribor6m)
    euribor6m_helpers.append(h)

# Swaps (discounted on the OIS curve)
sw_fixed_dc = ql.Thirty360(ql.Thirty360.European)

swap_quotes = {}  # keep SimpleQuote references for live bumping
swap_data = [
    ("3Y",  0.004240), ("4Y",  0.005760), ("5Y",  0.007620),
    ("6Y",  0.009540), ("7Y",  0.011350), ("8Y",  0.013030),
    ("9Y",  0.014520), ("10Y", 0.015840), ("12Y", 0.018090),
    ("15Y", 0.020370), ("20Y", 0.021870), ("25Y", 0.022340),
    ("30Y", 0.022560), ("35Y", 0.022950), ("40Y", 0.023480),
    ("50Y", 0.024210), ("60Y", 0.024630),
]
for tenor, rate in swap_data:
    q = ql.SimpleQuote(rate)
    swap_quotes[tenor] = q
    h = ql.SwapRateHelper(
        ql.QuoteHandle(q), ql.Period(tenor), calendar,
        ql.Annual, ql.Unadjusted, sw_fixed_dc, euribor6m,
        discountingCurve=discounting_handle,
    )
    euribor6m_helpers.append(h)

euribor6m_curve = ql.PiecewiseCubicDiscount(
    settlement_date, euribor6m_helpers, ts_day_counter)

forecasting_handle = ql.RelinkableYieldTermStructureHandle(euribor6m_curve)

print(f"Euribor 6M curve built with {len(euribor6m_helpers)} helpers")
print(f"  (1 deposit + {len(fra_data)} FRAs + {len(swap_data)} swaps)")

## Swap Pricing

Price a 5-year spot swap and a 1-year forward 5-year swap, both paying 0.70% fixed
vs Euribor 6M, discounted on the OIS curve.

In [ ]:
euribor_index = ql.Euribor6M(forecasting_handle)
swap_engine = ql.DiscountingSwapEngine(discounting_handle)

fixed_rate = 0.007
nominal = 1_000_000.0

# 5Y spot swap
maturity = calendar.advance(settlement_date, 5, ql.Years)
fixed_sched = ql.Schedule(
    settlement_date, maturity, ql.Period(ql.Annual), calendar,
    ql.Unadjusted, ql.Unadjusted, ql.DateGeneration.Forward, False,
)
float_sched = ql.Schedule(
    settlement_date, maturity, ql.Period(ql.Semiannual), calendar,
    ql.ModifiedFollowing, ql.ModifiedFollowing, ql.DateGeneration.Forward, False,
)
spot_swap = ql.VanillaSwap(
    ql.SwapType.Payer, nominal,
    fixed_sched, fixed_rate, ql.Thirty360(ql.Thirty360.European),
    float_sched, euribor_index, 0.0, ql.Actual360(),
)
spot_swap.setPricingEngine(swap_engine)

# 1Y forward 5Y swap
fwd_start = calendar.advance(settlement_date, 1, ql.Years)
fwd_maturity = calendar.advance(fwd_start, 5, ql.Years)
fwd_fixed_sched = ql.Schedule(
    fwd_start, fwd_maturity, ql.Period(ql.Annual), calendar,
    ql.Unadjusted, ql.Unadjusted, ql.DateGeneration.Forward, False,
)
fwd_float_sched = ql.Schedule(
    fwd_start, fwd_maturity, ql.Period(ql.Semiannual), calendar,
    ql.ModifiedFollowing, ql.ModifiedFollowing, ql.DateGeneration.Forward, False,
)
fwd_swap = ql.VanillaSwap(
    ql.SwapType.Payer, nominal,
    fwd_fixed_sched, fixed_rate, ql.Thirty360(ql.Thirty360.European),
    fwd_float_sched, euribor_index, 0.0, ql.Actual360(),
)
fwd_swap.setPricingEngine(swap_engine)

print(f"5Y spot swap paying {fixed_rate:.2%}")
print(f"1Y-forward 5Y swap paying {fixed_rate:.2%}")

## Results

In [ ]:
def print_swap_results(label, swap):
    print(f"  {label:30s}  NPV={swap.NPV():12.2f}  fair spread={swap.fairSpread():10.6%}  fair rate={swap.fairRate():10.6%}")

s5y = swap_quotes["5Y"]

print(f"With 5Y market swap rate = {s5y.value():.2%}")
print_swap_results("5Y swap paying 0.70%", spot_swap)
print_swap_results("1Y-fwd 5Y swap paying 0.70%", fwd_swap)

## Live Market Update

Bump the 5-year swap rate from 0.762% to 0.90%. Because the swap helpers hold
`SimpleQuote` references, changing the quote triggers automatic curve recalibration
and swap repricing. The forward swap is unaffected since it depends on different
rate pillars.

In [ ]:
s5y.setValue(0.0090)

print(f"With 5Y market swap rate = {s5y.value():.2%}")
print_swap_results("5Y swap paying 0.70%", spot_swap)
print_swap_results("1Y-fwd 5Y swap paying 0.70%", fwd_swap)